In [7]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import warnings

In [3]:
# Download historical VIX data

vix_data = yf.download("^VIX", start='2023-01-01', end='2024-01-01', progress=False)

vix_data.to_csv("VIX_prices.csv")

vix_data.head()

Price,Close,High,Low,Open,Volume
Ticker,^VIX,^VIX,^VIX,^VIX,^VIX
Date,,,,,
2023-01-03,22.900000,23.76,22.730000,23.090000,0
2023-01-04,22.010000,23.27,21.940001,22.930000,0
2023-01-05,22.459999,22.92,21.969999,22.200001,0
2023-01-06,21.129999,22.90,21.000000,22.690001,0
2023-01-09,21.969999,21.98,21.270000,21.750000,0


In [10]:
# Cleaning and scaling

from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")

scaler_vix = MinMaxScaler()

vix_data = vix_data[['Close']]

vix_data.fillna(method='ffill', inplace=True)

vix_data['Scaled Close'] = scaler_vix.fit_transform(vix_data['Close'])


In [14]:
# Sequences 

sequence_length = 10

def create_sequences(data, seq_length):
    sequences = []
    targets = []

    for i in range(len(data)-seq_length):
        sequences.append(data[i:i+seq_length])
        targets.append(data[i+seq_length])

    return np.array(sequences), np.array(targets)

X_vix, y_vix = create_sequences(vix_data['Close'].values, sequence_length)